In [1]:
import gc
import os
import pickle
import random
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
import polars as pl


In [5]:
class Config:
    PREPROCESS = False
    KAGGLE_NOTEBOOK = False
    DEBUG = True
    
    SEED = 42
    EPOCHS = 5
    BATCH_SIZE = 4096
    LR = 1e-3
    WD = 0.05
    PATIENCE = 5
    NBR_FOLDS = 15
    SELECTED_FOLDS = [0]
    
    
if Config.DEBUG:
    n_rows = 10**3
else:
    n_rows = None
    

In [19]:
if Config.KAGGLE_NOTEBOOK:
    RAW_DIR = "/kaggle/input/leash-BELKA/"
    PROCESSED_DIR = "/kaggle/input/belka-enc-dataset"
    OUTPUT_DIR = ""
    MODEL_DIR = ""
else:
    RAW_DIR = "../data/raw/"
    PROCESSED_DIR = "../data/processed/"
    OUTPUT_DIR = "../data/result/"
    MODEL_DIR = "../models/"

TRAIN_DATA_NAME = "local_train_enc.parquet"
SAVE_PATH = "../data/chuncked-dataset/"

データを10分割して保存

In [ ]:

# split dataset to several parquet
train = pl.read_parquet(os.path.join(PROCESSED_DIR, TRAIN_DATA_NAME), n_rows=None)

# shuffle
train = train.sample(fraction=1, seed=Config.SEED, shuffle=True)
# trainを10分割して保存
n = 10
chunk_size = len(train) // n
for i in range(n):
    chunk = train[i*chunk_size:(i+1)*chunk_size]
    chunk.write_parquet(os.path.join(SAVE_PATH, f"local_train_enc_{i}.parquet"))
    